<a href="https://colab.research.google.com/github/hiu04/Data-Science/blob/main/MMAI5200_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1

In [ ]:
def dijkstra(G, start, end):

    # Initialize the set of vertices Q and the distance array d
    unvisted = set(G.keys()) # All vertices in the graph G
    ## for all vertices v in unvisted, set the distance to infinite
    d = {v: float('infinity') for v in unvisted}
    ## staring point with distance of 0
    d[start] = 0
    ## store the predecessors, none for all vertices in initialization
    predecessors = {v: None for v in unvisted}


    while unvisted:
        current = None
        ## set min distance to infinity
        min_distance = float('infinity')
        ## Select the vertex current with the minimum d[u] value in unvisited
        for vertex in unvisted:
            if d[vertex] < min_distance:
                min_distance = d[vertex]
                current = vertex
        ## remove current vertex from unvisited
        unvisted.remove(current)

        # Break the loop if the smallest distance is infinity or
        # if the destination is found
        if d[current] == float('infinity') or current == end:
            break

        # Relax the edges from u to its neighbors
        for v, distance in G[current].items():
            if v in unvisted and d[current] + distance < d[v]:
                d[v] = d[current] + distance
                predecessors[v] = current

    # Reconstruct the shortest path from end to start
    path = []
    current_vertex = end
    while current_vertex is not None:
        path.insert(0, current_vertex)
        current_vertex = predecessors[current_vertex]

    return path, d[end]


graph = {
    'Origin': {'Vaughan': 20, 'Richmond Hill': 30, 'Markham': 25},
    'Vaughan': {'Richmond Hill': 5, 'North York': 35},
    'Richmond Hill': {'Richmond Hill': 10 , 'Markham': 25, 'North York': 20},
    'Markham': {'North York': 25, 'Destination': 50},
    'North York': {'Destination': 30},
    'Destination': {}}

shortest_distances = dijkstra(graph, 'Origin', 'Destination')
print("Shortest distances from Origin", ":", shortest_distances)


Shortest distances from Origin : (['Origin', 'Markham', 'Destination'], 75)


# Question 2

In [ ]:
#part (a)
import numpy as np

# States and rewards
states = [1, 2, 3]
profits = {1: 500, 2: 1000, 3: 3000}
advertising_cost = 100
discount_factor = 0.9

# Transition probabilities
transitions_no_ad = {
    1: {1: 0.4, 2: 0.6, 3: 0},
    2: {1: 0.3, 2: 0.4, 3: 0.3},
    3: {1: 0, 2: 0.6, 3: 0.4}
}

transitions_ad = {
    1: {1: 0.1, 2: 0.9, 3: 0},
    2: {1: 0.1, 2: 0.6, 3: 0.3},
    3: {1: 0, 2: 0.35, 3: 0.65}
}

# Value iteration function
def value_iteration(states, transitions_no_ad, transitions_ad, profits, advertising_cost, discount_factor, theta=0.0001):
    V = {s: 0 for s in states} # Initialize value for each state
    policy = {s: None for s in states} # Initialize policy

    while True:
        delta = 0
        for s in states:
            v = V[s]

            # Calculate the value of not advertising and advertising
            no_ad_value = sum(transitions_no_ad[s][s_prime] * (profits[s_prime] + discount_factor * V[s_prime]) for s_prime in states)
            ad_value = sum(transitions_ad[s][s_prime] * (profits[s_prime] - advertising_cost + discount_factor * V[s_prime]) for s_prime in states)

            # Choose the best action
            V[s] = max(no_ad_value, ad_value)

            delta = max(delta, abs(v - V[s]))
        if delta < 0.05: # A small threshold for convergence
            break

    # Derive policy from value function
    for s in states:
        no_ad_value = sum(transitions_no_ad[s][s_prime] * (profits[s_prime] + discount_factor * V[s_prime]) for s_prime in states)
        ad_value = sum(transitions_ad[s][s_prime] * (profits[s_prime] - advertising_cost + discount_factor * V[s_prime]) for s_prime in states)

        policy[s] = 'A' if ad_value > no_ad_value else 'N'

    return policy, V

policy, V = value_iteration(states, transitions_no_ad, transitions_ad, profits, advertising_cost, discount_factor)

print("Optimal policy:", policy)
print("Value function:", V)


Optimal policy: {1: 'A', 2: 'A', 3: 'A'}
Value function: {1: 16021.006292419286, 2: 16949.5764974934, 3: 18166.485062498927}


In [ ]:
#part (b)
import numpy as np

# Define the states and corresponding profits
states = [1, 2, 3]
profits = {1: 500, 2: 1000, 3: 3000}
advertising_cost = 100
discount_factor = 0.9

# Transition probabilities for not advertising
transition_no_ad = {
    1: {1: 0.4, 2: 0.6, 3: 0},
    2: {1: 0.3, 2: 0.4, 3: 0.3},
    3: {1: 0, 2: 0.6, 3: 0.4}
}

# Transition probabilities for advertising
transition_ad = {
    1: {1: 0.1, 2: 0.9, 3: 0},
    2: {1: 0.1, 2: 0.6, 3: 0.3},
    3: {1: 0, 2: 0.35, 3: 0.65}
}

# Policy iteration algorithm
def policy_iteration(states, transition_no_ad, transition_ad, profits, advertising_cost, discount_factor):
    # Initial policy is not to advertise in any state
    policy = {state: 'N' for state in states}
    V = {state: 0 for state in states}

    is_policy_stable = False
    while not is_policy_stable:
        # Policy Evaluation
        while True:
            delta = 0
            for s in states:
                v = V[s]
                # Calculate value for current policy
                transition_prob = transition_ad[s] if policy[s] == 'A' else transition_no_ad[s]
                V[s] = sum(transition_prob[s_prime] * (profits[s_prime] - (advertising_cost if policy[s] == 'A' else 0) \
                                                       + discount_factor * V[s_prime]) for s_prime in states)
                delta = max(delta, abs(v - V[s]))
            if delta < 0.05: # A small threshold for convergence
                break

        # Policy Improvement
        is_policy_stable = True
        for s in states:
            old_action = policy[s]

            # Check if advertising in state s is better
            value_no_ad = sum(transition_no_ad[s][s_prime] * (profits[s_prime] + discount_factor * V[s_prime]) for s_prime in states)
            value_ad = sum(transition_ad[s][s_prime] * (profits[s_prime] - advertising_cost + discount_factor * V[s_prime]) for s_prime in states)

            policy[s] = 'A' if value_ad > value_no_ad else 'N'

            if old_action != policy[s]:
                is_policy_stable = False

    return policy, V

policy, V = policy_iteration(states, transition_no_ad, transition_ad, profits, advertising_cost, discount_factor)

print("Optimal policy:", policy)
print("Value function:", V)


Optimal policy: {1: 'A', 2: 'A', 3: 'A'}
Value function: {1: 16021.01748180916, 2: 16949.587366370146, 3: 18166.495350059486}
